# Planetary Motion

This little simulation was inspired by hearing that if we want to travel to mars, there is a special window every 2 years or so that makes it easier. So I wanted to understand why that is.

All that is needed to simulate how the planets move around the sun are the masses and positions/velocities at any given point in time. Then Newton's Law of Universal Gravitation takes care of the rest.

Estimates of planet data is available at [NASA](https://nssdc.gsfc.nasa.gov/planetary/planetfact.html).

For someone not fluent in Astronomy it can be quite tricky to interpret those fact sheets. Atleast 'mass' is there in plain text. I decided to use the following variables to construct a position and velocity of all planets at a specific point in time.

1. Mass
2. Perihelion
3. MaxVelocity
4. Orbital inclination
5. Longitude of ascending node
6. Longitude of perihelion

(Longitude of perihelion is actually a sum of two different angles that point in different directions! It is really the 'argument of periapsis' I want which is equal to Longitude of perihelion minus Longitude of ascending node)

In [128]:
m = [1.9885e30, 0.33011e24, 4.8675e24, 5.9724e24, 0.64171e24, 1898.19e24, 568.34e24, 86.813e24, 102.413e24]
peri = [0, 46.00e9, 107.48e9, 147.09e9, 206.62e9, 740.52e9, 1352.55e9, 2741.30e9, 4444.45e9]
vp    = [0, 58.98e3, 35.26e3, 30.29e3, 26.50e3, 13.72e3, 10.18e3, 7.11e3, 5.50e3]
incl = π/180*[0, 7.00487, 3.39471, 0.00005, 1.85061, 1.30530, 2.48446, 0.76986, 1.76917]
loan = π/180*[0, 48.33167, 76.68069, -11.26064, 49.57854, 100.55615, 113.71504, 74.22988, 131.72169]
lop  = π/180*[0, 77.45645, 131.53298, 102.94719, 336.04084, 14.75385, 92.43194, 170.96424, 44.97135];

In [129]:
function angles2rotm(a, b, c)
    #Euler_angle_transformations
    #a=Longitude of ascending node, b=Inclination, c=Argument of periapsis
    R=zeros(3,3)
    R[:,1] = [cos(a)*cos(c) - sin(a)*cos(b)*sin(c), -cos(a)*sin(c) - sin(a)*cos(b)*cos(c), sin(b)*sin(a)]
    R[:,2] = [sin(a)*cos(c) + cos(a)*cos(b)*sin(c), -sin(a)*sin(c) + cos(a)*cos(b)*cos(c), -sin(b)*cos(a)]
    R[:,3] = [sin(b)*sin(c), sin(b)*cos(c), cos(b)]
    return R
end

angles2rotm (generic function with 1 method)

In [130]:
#make position/velocity vectors by rotating perihelion position/velocity in 3d
function place_planets(peri, vp, loan, incl, lop)
    p = zeros(9,3)
    v = zeros(9,3)
    for n=1:9
        R=angles2rotm(loan[n], incl[n], lop[n]-loan[n])
        p[n,:] = R*[peri[n],0,0]
        v[n,:] = R*[0,vp[n],0]
    end
    return p,v
end

In [131]:
function Force(m, p, b, a)
    #forcevector felt on b from a as function of their mass and position
    G = 6.674e-11 #gravitational constant
    d = p[b,:]-p[a,:]
    return -d .* G*m[a]*m[b]*(d'*d)^(-3/2)
end

Force (generic function with 1 method)

In [132]:
using Plots
#gr(size=(800,600))
gr()

Plots.GRBackend()

In [ ]:
#position,velocity,mass
function simulate_solar_system(p,v,m)
    F = zeros(9,3)
    Δt = 24*60*60
    #simulationlength = 365*100 #total simulation length = simulationlength*Δt seconds
    simulationlength = 365*30

    ss=peri[5]*1.5 #size of plot
    dist_earth_to_planet = zeros(simulationlength, 9)

    @gif for t=1:simulationlength
        # calculate forces, accelerations, velocities and update positions
        for n=2:9; F[n,:] = Force(m, p, n, 1) end
        a=F./m
        v += a.*Δt
        p += v.*Δt

        for planet=1:9
            dist_earth_to_planet[t, planet] = norm(p[planet,:]-p[4,:])/1e9
        end

        # visualize
        if t%5==0
            # plot the positions in 2d even though everyhing is in 3d because its easier to see
            p1 = scatter(p[:,1],p[:,2], xlim=(-ss,ss), ylim=(-ss,ss), leg=false, xtick=false, ytick=false, title="Inner Solar System", axis=false)
            plot!(p1, [p[4,1];p[5,1]], [p[4,2];p[5,2]]) #add a line between earth and mars

            # plot the earth-mars distance over time

            p2 = plot(dist_earth_to_planet[1:t,5], ylim=(2e10/1e9, 5e11/1e9), title="Distance Earth-Mars (in million kilometers)", xlab="days")
            plot(p1,p2, layout=grid(2,1,heights=[0.8,0.2]), leg=false)
        end

    end every 5
    return dist_earth_to_planet
end

<img src="tmp.gif">

In [143]:
function avgperiod(e2p)
    binaray_toward_away = e2p[2:end].>e2p[1:end-1]
    index_of_minima = find(binaray_toward_away[2:end].>binaray_toward_away[1:end-1])
    periods = index_of_minima[2:end].-index_of_minima[1:end-1]
    avgperiod_in_days = mean(periods[2:end-1])
    return avgperiod_in_days
end

avgperiod (generic function with 1 method)

In [167]:
period = avgperiod(dist_earth_to_planet[:,5])
println("average period of earth opposition with mars: ", Int(round(period)), " days")

average period of earth opposition with mars: 778 days


In [166]:
period = avgperiod(dist_earth_to_planet[:,3])
println("average period of earth opposition with venus: ", Int(round(period)), " days")

average period of earth opposition with venus: 585 days
